In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional,Embedding
import re

In [2]:
data = pd.read_csv('data.csv')

print(data.head())

   id                                           consulta          intencion
0   1   ¿Cuál es la fecha límite para el proyecto Final?     fechas_entrega
1   2   ¿Cuáles son los temas a tratar en la clase de...            temario
2   3   ¿Dónde puedo encontrar material de estudio ad...   material_estudio
3   4   ¿Cuál es el horario de consulta del profesor ...           horarios
4   5                     ¿Cómo implemento un árbol AVL?    ejemplos_codigo


### Preparacion del Dataset

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         102 non-null    int64 
 1   consulta   102 non-null    object
 2   intencion  102 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


In [4]:
data.columns

Index(['id', 'consulta', 'intencion'], dtype='object')

In [5]:
# removiendo el espacio de las columnas
data.columns = data.columns.str.strip()
data['consulta'] = data['consulta'].str.strip()
data['intencion'] = data['intencion'].str.strip()

print(data.values)

[[1 '¿Cuál es la fecha límite para el proyecto Final?' 'fechas_entrega']
 [2 '¿Cuáles son los temas a tratar en la clase de esta semana?'
  'temario']
 [3 '¿Dónde puedo encontrar material de estudio adicional?'
  'material_estudio']
 [4
  '¿Cuál es el horario de consulta del profesor de Programación Avanzada?'
  'horarios']
 [5 '¿Cómo implemento un árbol AVL?' 'ejemplos_codigo']
 [6 '¿Hay alguna práctica programada para el próximo fin de semana?'
  'horarios']
 [7
  '¿Dónde puedo encontrar ejemplos de código para algoritmos de ordenamiento eficientes?'
  'ejemplos_codigo']
 [8 '¿Cuál es el cronograma de exámenes para este semestre?' 'cronograma']
 [10 '¿Cuándo es la entrega de la implementación de árboles rojinegros?'
  'fechas_entrega']
 [11
  '¿Dónde puedo encontrar información sobre la solución del ejercicio de backtracking?'
  'material_estudio']
 [12
  '¿Hay algún repositorio de código disponible para la práctica de pilas y colas en Python?'
  'repositorio']
 [13
  '¿Cómo puedo ha

In [6]:
#sort data by intencion
data = data.sort_values(by=['intencion'], ascending=True)
print(data.head())

    id                                           consulta   intencion
50  52  ¿Dónde puedo obtener una copia del cronograma ...  cronograma
43  45  ¿Dónde puedo encontrar el cronograma de exámenes?  cronograma
28  30  ¿Dónde puedo obtener una copia del cronograma ...  cronograma
27  29  ¿Cuál es el cronograma de entregas para las ta...  cronograma
52  54  ¿Dónde puedo encontrar el cronograma de exámenes?  cronograma


In [7]:
# Eliminando los signos de puntuacion
data['consulta'] = data['consulta'].str.replace(r'[^\w\s]', '', regex=True)

In [8]:
train_txt,test_txt,train_label,test_labels = train_test_split(data['consulta'],data['intencion'],test_size = 0.2)

In [9]:
#guardar los datos de entrenamiento
train_label.to_csv('train_label.csv',index=False)

### Tokenizacion

In [10]:
# Convertir el texto en una lista de palabras
words = []
for text in data['consulta']:
    words += text_to_word_sequence(text)

# Contar el número de palabras únicas
unique_words = set(words)
num_unique_words = len(unique_words)

# Imprimir el número de palabras únicas
print("Número de palabras únicas: ", num_unique_words)

Número de palabras únicas:  211


In [11]:
max_num_words = 350 #el numero de palabras unicas es de 211 entonces agregamos un maximo con holgura
classes = np.unique(data['intencion'])

tokenizer = Tokenizer(num_words=max_num_words)
tokenizer.fit_on_texts(train_txt)
word_index = tokenizer.word_index

word_index

{'de': 1,
 'el': 2,
 'la': 3,
 'puedo': 4,
 'dónde': 5,
 'encontrar': 6,
 'para': 7,
 'es': 8,
 'del': 9,
 'cuál': 10,
 'horario': 11,
 'los': 12,
 'en': 13,
 'pa': 14,
 'las': 15,
 'fecha': 16,
 'repositorio': 17,
 'límite': 18,
 'cronograma': 19,
 'ejemplos': 20,
 'entrega': 21,
 'temario': 22,
 'programación': 23,
 'que': 24,
 'este': 25,
 'temas': 26,
 'se': 27,
 'semana': 28,
 'práctica': 29,
 'avanzada': 30,
 'proyecto': 31,
 'código': 32,
 'final': 33,
 'con': 34,
 'cuáles': 35,
 'son': 36,
 'específicos': 37,
 'sesiones': 38,
 'cuatrimestre': 39,
 'necesito': 40,
 'al': 41,
 'actualizado': 42,
 'árboles': 43,
 'tp': 44,
 'obtener': 45,
 'una': 46,
 'copia': 47,
 'esta': 48,
 'quiero': 49,
 'próximo': 50,
 'tutoría': 51,
 'hay': 52,
 'examen': 53,
 'información': 54,
 'sobre': 55,
 'clases': 56,
 'repaso': 57,
 'trabajo': 58,
 'algoritmos': 59,
 'mes': 60,
 'cómo': 61,
 'exámenes': 62,
 'presentación': 63,
 'curso': 64,
 'cuándo': 65,
 'consulta': 66,
 'tenes': 67,
 'enlace': 68

In [12]:
ls=[]
for c in train_txt:
    ls.append(len(c.split()))   #cada consulta se convierte en una lista de palabras y se cuenta el numero de palabras

maxLen=int(np.percentile(ls, 98))   #se calcula el percentil 98 de la lista de palabras

train_sequences = tokenizer.texts_to_sequences(train_txt)   #convierte las consultas en secuencias de tokens
print("train sequence tokenice:\n",train_sequences)

train_sequences = pad_sequences(train_sequences, maxlen=maxLen, padding='post')     #rellena con ceros las secuencias para que todas tengan la misma longitud dada por el percentil 98
print("train sequence pad sequence:\n",train_sequences)

test_sequences = tokenizer.texts_to_sequences(test_txt)
print("test sequence tokenice:\n",test_sequences)
test_sequences = pad_sequences(test_sequences, maxlen=maxLen, padding='post')
print("test sequence tokenice:\n",test_sequences)

train sequence tokenice:
 [[52, 69, 70, 1, 94, 71, 95, 9, 53, 33, 1, 3, 96], [5, 4, 6, 2, 11, 1, 15, 38, 1, 97, 7, 3, 72, 70, 1, 98, 1, 99, 34, 100, 101], [10, 8, 3, 16, 18, 7, 3, 21, 9, 44], [5, 4, 6, 54, 55, 15, 102, 1, 21], [5, 4, 6, 103, 104, 7, 105, 12, 73, 1, 23, 106], [5, 4, 45, 46, 47, 9, 19, 1, 107], [35, 36, 12, 26, 37, 24, 27, 108, 48, 28], [10, 8, 2, 11, 29, 1, 48, 28], [49, 24, 109, 110, 2, 22, 1, 25, 39], [35, 36, 12, 26, 111, 112, 13, 3, 74, 1, 48, 28], [5, 4, 6, 2, 11, 1, 15, 56, 1, 57, 7, 2, 53, 33], [35, 36, 12, 26, 37, 24, 27, 113, 13, 2, 22], [114, 8, 2, 17, 1, 75, 30], [40, 76, 77, 1, 78], [40, 2, 115, 41, 17, 1, 14], [5, 4, 6, 2, 22, 42, 34, 15, 116, 1, 25, 39], [10, 8, 3, 16, 18, 7, 3, 21, 58, 79, 117, 118], [35, 36, 12, 26, 37, 24, 27, 80, 13, 2, 22, 1, 3, 119, 28], [49, 20, 1, 120, 1, 59, 1, 81], [5, 4, 6, 20, 1, 121, 9, 122, 123, 13, 3, 23, 1, 59, 124], [10, 8, 3, 16, 18, 7, 3, 21, 44, 125], [5, 4, 6, 20, 1, 126, 1, 43, 82, 13, 2, 127, 128], [5, 4, 6, 54, 55, 

In [13]:
label_encoder = LabelEncoder() # codifica las clases en numeros
integer_encoded = label_encoder.fit_transform(classes) # codifica las clases en numeros

print("Clases: ", classes)
print("Clases codificadas: ", integer_encoded)

Clases:  ['cronograma' 'ejemplos_codigo' 'fechas_entrega' 'horarios'
 'material_estudio' 'repositorio' 'temario']
Clases codificadas:  [0 1 2 3 4 5 6]


In [14]:
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

print(integer_encoded)

[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]]


In [15]:
onehot_encoder.fit(integer_encoded)

print("Onehot catgories:",onehot_encoder.categories_) # codifica las clases en numeros

Onehot catgories: [array([0, 1, 2, 3, 4, 5, 6])]


c:\Users\Marcos\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [16]:
# dimesiones del onehot_encoder
# num_classes = len(onehot_encoder.categories_[0])
# print("Número de clases: ", num_classes)
# print("Clases \n", onehot_encoder.categories_[0])

In [17]:
train_label

15            horarios
63            horarios
51      fechas_entrega
33      fechas_entrega
19    material_estudio
            ...       
76          cronograma
43          cronograma
88         repositorio
10         repositorio
71            horarios
Name: intencion, Length: 81, dtype: object

In [18]:
train_label_encoded = label_encoder.transform(train_label) # codifica las clases en numeros

train_label_encoded

array([3, 3, 2, 2, 4, 0, 6, 3, 6, 6, 3, 6, 5, 4, 3, 6, 2, 6, 1, 1, 2, 1,
       6, 0, 6, 4, 4, 3, 4, 0, 0, 5, 3, 2, 2, 2, 3, 2, 2, 3, 4, 5, 1, 3,
       5, 6, 1, 2, 3, 3, 6, 6, 2, 6, 5, 3, 3, 0, 0, 3, 3, 1, 3, 5, 2, 3,
       2, 0, 5, 4, 5, 5, 0, 4, 2, 3, 0, 0, 5, 5, 3])

In [19]:
train_label_encoded = train_label_encoded.reshape(len(train_label_encoded), 1)

train_label_encoded

array([[3],
       [3],
       [2],
       [2],
       [4],
       [0],
       [6],
       [3],
       [6],
       [6],
       [3],
       [6],
       [5],
       [4],
       [3],
       [6],
       [2],
       [6],
       [1],
       [1],
       [2],
       [1],
       [6],
       [0],
       [6],
       [4],
       [4],
       [3],
       [4],
       [0],
       [0],
       [5],
       [3],
       [2],
       [2],
       [2],
       [3],
       [2],
       [2],
       [3],
       [4],
       [5],
       [1],
       [3],
       [5],
       [6],
       [1],
       [2],
       [3],
       [3],
       [6],
       [6],
       [2],
       [6],
       [5],
       [3],
       [3],
       [0],
       [0],
       [3],
       [3],
       [1],
       [3],
       [5],
       [2],
       [3],
       [2],
       [0],
       [5],
       [4],
       [5],
       [5],
       [0],
       [4],
       [2],
       [3],
       [0],
       [0],
       [5],
       [5],
       [3]])

In [20]:
train_label = onehot_encoder.transform(train_label_encoded)    # codifica las clases en numeros

train_label

array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
 

In [21]:
#ahora lo mismo pero para los datos de testeo

test_labels_encoded = label_encoder.transform(test_labels)
test_labels_encoded = test_labels_encoded.reshape(len(test_labels_encoded), 1)

print("pre-onehot:\n",test_labels)

test_labels = onehot_encoder.transform(test_labels_encoded)

print("\n\npos-onehot:\n",test_labels)

pre-onehot:
 54       fechas_entrega
91              temario
87           cronograma
39           cronograma
83             horarios
85              temario
53          repositorio
52           cronograma
45          repositorio
64             horarios
99           cronograma
47       fechas_entrega
27           cronograma
18     material_estudio
36              temario
81       fechas_entrega
101    material_estudio
16      ejemplos_codigo
98              temario
24             horarios
38           cronograma
Name: intencion, dtype: object


pos-onehot:
 [[0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 

### Entrenando el modelo de Word2Vec

In [22]:
words

['dónde',
 'puedo',
 'obtener',
 'una',
 'copia',
 'del',
 'cronograma',
 'de',
 'prácticas',
 'dónde',
 'puedo',
 'encontrar',
 'el',
 'cronograma',
 'de',
 'exámenes',
 'dónde',
 'puedo',
 'obtener',
 'una',
 'copia',
 'del',
 'cronograma',
 'de',
 'evaluaciones',
 'para',
 'el',
 'próximo',
 'año',
 'académico',
 'cuál',
 'es',
 'el',
 'cronograma',
 'de',
 'entregas',
 'para',
 'las',
 'tareas',
 'restantes',
 'del',
 'semestre',
 'dónde',
 'puedo',
 'encontrar',
 'el',
 'cronograma',
 'de',
 'exámenes',
 'dónde',
 'puedo',
 'obtener',
 'una',
 'copia',
 'del',
 'cronograma',
 'de',
 'practicas',
 'de',
 'pa',
 'programación',
 'avanzada',
 'dónde',
 'puedo',
 'encontrar',
 'el',
 'cronograma',
 'de',
 'exámenes',
 'dónde',
 'puedo',
 'encontrar',
 'el',
 'cronograma',
 'actualizado',
 'de',
 'evaluaciones',
 'de',
 'programacion',
 'avanzada',
 'cuál',
 'es',
 'el',
 'cronograma',
 'de',
 'pa',
 'de',
 'este',
 'cuatrimestre',
 'dónde',
 'puedo',
 'obtener',
 'una',
 'copia',
 'de

In [23]:
w2v_model = Word2Vec(sentences=[words], vector_size=100, window=5, min_count=1, workers=4)
"""
    Parametros:
    sentences: lista de lista de palabras
    size: dimension del vector (posible parametro a modificar 100-300)
    window: tamaño de la ventana de palabras (ventana: palabras que se encuentran a los lados de la palabra objetivo)
    min_count: minimo de veces que debe aparecer una palabra para ser considerada (es un dataset chico asi que se deja en 1)
    workers: numero de hilos de ejecucion
"""


'\n    Parametros:\n    sentences: lista de lista de palabras\n    size: dimension del vector (posible parametro a modificar 100-300)\n    window: tamaño de la ventana de palabras (ventana: palabras que se encuentran a los lados de la palabra objetivo)\n    min_count: minimo de veces que debe aparecer una palabra para ser considerada (es un dataset chico asi que se deja en 1)\n    workers: numero de hilos de ejecucion\n'

In [24]:
# mostrar las dimensiones de la matriz de embeddings
print("Dimensiones de la matriz de embeddings: ", w2v_model.wv.vectors.shape)

Dimensiones de la matriz de embeddings:  (211, 100)


In [25]:
# mostramos los vectores de las palabras
print(w2v_model.wv.vectors[1])

[-0.01076306  0.00718829  0.00408915  0.00642215  0.00866755 -0.01304841
  0.00412582  0.0141335  -0.00699785 -0.00908607 -0.00165412 -0.0137612
 -0.00592462  0.0096036   0.00455162  0.00379571  0.00856021  0.00500713
 -0.00490827 -0.00973071  0.00302717 -0.0022888   0.01104611 -0.01068563
  0.00604263  0.00342569 -0.00686546  0.00341924 -0.00484632  0.00686655
  0.01391384 -0.00436521  0.00265058 -0.01018085  0.00103897  0.00892116
  0.00986585  0.00460085  0.00862124  0.00404157  0.00941136 -0.01202924
 -0.01113608 -0.00191462  0.00051403  0.00865595  0.0025814  -0.00248957
  0.00399516  0.0031886   0.00960894 -0.0115593   0.00103191  0.00302574
 -0.00141472  0.00918643  0.01154194  0.00586984 -0.00407728  0.00939944
 -0.00758446  0.00232315 -0.00297465 -0.00592903  0.00166015  0.00957097
  0.00843098 -0.00079913  0.00111493  0.01165282 -0.00678038 -0.00704475
  0.00908658  0.00675251  0.00394422 -0.00549796 -0.00782748 -0.00031667
  0.00254652 -0.00385612 -0.01217128  0.0049877   0.

In [26]:
# mostramos los embeddings de la palabra "hola"
print(w2v_model.wv['dónde'])

[-9.2231408e-03  4.7834730e-03 -8.2930280e-03 -1.6092287e-03
  5.0338195e-03 -1.0380825e-03  4.1080848e-03  1.0306911e-02
 -8.2265800e-03  4.3195765e-03 -7.4328827e-03 -7.7517319e-04
 -8.4470846e-03  4.5720413e-03 -3.8525849e-03 -7.4047204e-03
 -1.4502687e-03  3.1906622e-03  4.7205286e-03 -1.3973443e-02
  1.4518070e-03 -6.2836739e-03  1.0256744e-02  8.5055986e-03
 -3.2795351e-03  1.2684301e-03 -1.1400800e-03  4.3563684e-03
 -1.1708008e-02  6.6899665e-04  1.0694679e-02  2.4324651e-03
  4.3680011e-03 -1.3705671e-02  5.6707393e-03 -1.5508276e-04
  1.3191091e-04  1.8351773e-03 -1.5997282e-03 -3.9339345e-03
  3.1392740e-03 -1.1795669e-02 -1.1673977e-02  7.2277579e-03
  9.7305411e-03 -6.1917901e-03  6.9202455e-05 -7.9927000e-04
  7.1327738e-03 -5.8215763e-03  5.8255824e-03  2.3192461e-03
  1.0962043e-02 -4.7695665e-03 -1.5980708e-03 -6.6363574e-03
 -7.1133762e-03 -9.8364791e-03 -4.3229843e-03 -4.8678573e-03
  5.6163482e-03 -6.9436939e-03  4.2905966e-03 -1.2481523e-04
 -5.3811269e-03  3.26550

In [27]:
# mostramos el tamaño del vocabulario
print("Tamaño del vocabulario: ", len(w2v_model.wv))

Tamaño del vocabulario:  211


### Creacion del modelo (RNN-LSTM)

**A. Numero de Neuronas:**

&nbsp;&nbsp;&nbsp;&nbsp; Para calcular las neuronas de la capa de LSTM existen 2 formulas:

1)  $N_h = \frac{N_s}{α * (N_i + N_o)}$

2)  $N_h = \frac{2}{3} * (N_i+N_o)$


**B. Numero de Capas:**

&nbsp;&nbsp;&nbsp;&nbsp;En este caso decidi usar solo una capa LSTM bidireccional y una capa densa.

&nbsp;&nbsp;&nbsp;&nbsp;Adicionalmente se debe acompañar con una capa de dropout para evitar el sobreajuste. La idea de esta capa es ignorar neuronas seleccionadas aleatoriamente durante el entrenamiento. Segun he leido con un 20% se puede lograr un equilibrio entre presicion y evite de sobreajuste.

*actualizacion*

&nbsp;&nbsp;&nbsp;&nbsp;**tuve que agregar una capa unidireccional porque la complejidad de los datos no me permitia entrenarlos con tan solo una capa**

**C. Funcion de activacion:**

&nbsp;&nbsp;&nbsp;&nbsp;En cuanto a la funcion de activacion, en nuestro problema, tenemos varias clases, pero sólo una de ellas puede estar presente al mismo tiempo. Para este tipo de problemas, por lo general, la función de activación softmax funciona mejor, ya que nos permite interpretar las salidas como probabilidades.

In [28]:
modelo_LSTM = tf.keras.Sequential()



# modelo_LSTM.add(Embedding(input_dim=num_unique_words, output_dim=100, input_length=maxLen, trainable=False, weights=[w2v_model.wv.vectors]))
modelo_LSTM.add(Embedding(input_dim=num_unique_words, output_dim=100, input_length=train_sequences.shape[1], trainable=False, weights=[w2v_model.wv.vectors]))
"""
    Parametros:
    input_dim: tamaño del vocabulario (numero de palabras unicas)
    output_dim: tamaño del vector de salida (dimension del vector)
    input_length: longitud de la secuencia de entrada (numero de palabras en cada consulta)

"""
modelo_LSTM.add(Bidirectional(LSTM(71, return_sequences=True, dropout=0.1, recurrent_dropout=0.1), 'concat')) #Probablemente pueda usar 71, segun el calculo de la formula: 2/3*(n_input+n_output)
"""
    Parametros:
    units: dimension de la salida del LSTM (numero de neuronas)
    return_sequences: si la salida es una secuencia o un vector (True: secuencia, False: vector)
    dropout: porcentaje de neuronas que se desactivan en cada iteracion (evita el overfitting)
    recurrent_dropout: porcentaje de neuronas que se desactivan en cada iteracion de la capa recurrente (evita el overfitting)

    concat: concatena las salidas de las capas recurrentes (bidireccional)
"""
modelo_LSTM.add(Dropout(0.2))
modelo_LSTM.add(LSTM(71, dropout=0.1, recurrent_dropout=0.1, return_sequences=False))
"""
    esta capa cumple la funcion de reducir la dimension de la salida de la capa anterior
    esto se debe a que la capa anterior devuelve una secuencia de vectores y la siguiente capa espera un vector
"""
modelo_LSTM.add(Dropout(0.2)) # desactiva el 20% de las neuronas

# modelo_LSTM.add(Dense(64, activation='relu')) # Aca se usa la mitad de las neuronas de la capa anterior
"""
    Parametros:
    units: dimension de la salida de la capa, es 64 porque es la mitad de la salida de la capa anterior
    activation: funcion de activacion
"""
modelo_LSTM.add(Dense(7)) # Esta es la capa de salida, va a tener tantas neuronas como clases haya
"""
    Parametros:
    units: dimension de la salida de la capa, es num_classes porque es el numero de clases
    activation: funcion de activacion (softmax porque es un problema de clasificacion)
"""
modelo_LSTM.add(Activation('softmax')) # funcion de activacion softmax

modelo_LSTM.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

### Entremiendo del Modelo

**Tamaño del lote (batch_size)**

&nbsp;&nbsp;&nbsp;&nbsp;Este numero lo calcule en base a la cantidad de datos de entrenamiento, podria usar 8, 16 o 32

**Epocas (epochs)**

&nbsp;&nbsp;&nbsp;&nbsp;comence con 10, puedo ir probando

In [30]:
# Entrenamiento del modelo
modelo_LSTM.fit(train_sequences, train_label, epochs=10, batch_size=16 , validation_data=(test_sequences, test_labels))

Epoch 1/10
6/6 [==============================] - 9s 151ms/step - loss: 1.9440 - acc: 0.1728 - val_loss: 1.9469 - val_acc: 0.1429
Epoch 2/10
6/6 [==============================] - 0s 32ms/step - loss: 1.9320 - acc: 0.2469 - val_loss: 1.9346 - val_acc: 0.1429
Epoch 3/10
6/6 [==============================] - 0s 32ms/step - loss: 1.9249 - acc: 0.2469 - val_loss: 1.9328 - val_acc: 0.1429
Epoch 4/10
6/6 [==============================] - 0s 33ms/step - loss: 1.9024 - acc: 0.2469 - val_loss: 1.9443 - val_acc: 0.1429
Epoch 5/10
6/6 [==============================] - 0s 33ms/step - loss: 1.8920 - acc: 0.2469 - val_loss: 1.9949 - val_acc: 0.1429
Epoch 6/10
6/6 [==============================] - 0s 34ms/step - loss: 1.8924 - acc: 0.2469 - val_loss: 1.9741 - val_acc: 0.1429
Epoch 7/10
6/6 [==============================] - 0s 33ms/step - loss: 1.9149 - acc: 0.2469 - val_loss: 1.9753 - val_acc: 0.1429
Epoch 8/10
6/6 [==============================] - 0s 33ms/step - loss: 1.8995 - acc: 0.2469 - va